## solve linear equations

### how do you describe and compute the vectors in the spaces?

turning the idea into an algorithm - what is the algorithm for
solving $Ax = 0$

example:

$A = \begin{bmatrix}
1 & 2 & 2 & 2 \\
2 & 4 & 6 & 8 \\
3 & 6 & 8 & 10
\end{bmatrix}$




In [8]:
import pylinearalgebra.mattools as mt
A = [
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
]
R, i_pivots = mt.rref(A)
mt.pprint(R)
print('pivot columns (x1, x3):', i_pivots)
print('free columns: (x2, x4):', [n for n in range(4) if n not in i_pivots])

⎡  1   2   0  -2 ⎤
⎢  0   0   1   2 ⎥
⎣  0   0   0   0 ⎦
pivot columns (x1, x3): [0, 2]
free columns: (x2, x4): [1, 3]


column-2 and row-3 are not independent, because

column-2 is a multiple of column-1

row-3 is the sum of row-1 and row-2 

the vectors of these are in the same direction

we extend the algorithm of elimination to cover the case where
the pivot is zero; we just move on

while doing the elimination **the nullspace is unchanged**. but we
are chaning the column space.


after the first step of elimination, $A$ becomes

$\begin{bmatrix}
1 & 2 & 2 & 2 \\
0 & 0 & 2 & 4 \\
0 & 0 & 2 & 4
\end{bmatrix}$

the pivot position $A_{22}$ is zero, and the position below it
$A_{32}$ is also zero,

it a telling sign that column-2 depends on an earlier column (hence
the two vertical zeros)

therefore we move on to the next good pivot, $A_{23}$

$\begin{bmatrix}
1 & 2 & 2 & 2 \\
0 & 0 & 2 & 4 \\
0 & 0 & 0 & 0
\end{bmatrix}$

this is $U$, or upper echelon form

source: <https://en.wikipedia.org/wiki/Row_echelon_form>



### the rank of a matrix

the number of the pivots, $2$ is the rank of this matrix, $A$

the columns these pivots reside are **pivot columns**

we transformed $Ax = 0$ to $Ux = 0$ while preserving the solutions
and nullspace

the other columns are called **free columns** - because the
variables associated with these columns can hold any values

for example I can assign $x_{2}, x_{4}$ with 
$\begin{bmatrix}1 \\ 0 \end{bmatrix}$ - the choice zero is purely
for convenience (as any value works)



the algorithm to get the solutions is to separate the **pivot
columns**

plug $x_{2}$ and $x_{4}$ in the remaining equations:

$x_{1} + x_2{} * 2 + x_{3} * 2 + x_{4} * 2 = 0 \\
x_{3} * 2 + x_{4} * 4 = 0
$

we solve $x_{1}, x_{3}$ `(-2, 0)` and get the solution

$C = \begin{bmatrix}
-2 \\ 1 \\ 0 \\ 0
\end{bmatrix}$

#### where are the other solutions ?

1) we can take multiples of this particular solution, $C$, forming
a line in this 4-D space

2) assign other values (such as `(0, 1)`) to the free variables $x_{2}, x_{4}$


$\begin{align}
X = c\begin{bmatrix}-2 \\ 1 \\ 0 \\ 0\end{bmatrix} +
d\begin{bmatrix}2 \\ 0 \\ -2 \\ 1\end{bmatrix}
\end{align}$

the "special" values `(1, 0), (0, 1)` are chosen, then the rest
of the solutions of the nullspace are the linear combinations of
these two solutions, which solve $Ux = 0$


#### rank and the number of solutions

in the above case $A$, $r = 2$, meaning that there are really just
2 equations. Therefore the number of pivot columns is 2 and 
the number of free vars is $4 - 2 = 2$

generalising this to:

for an $n \times m$ matrix, the free variables are $n - r$


### reduced row echelon form, R

recall that the row echelon form of $A$ is

$A = \begin{bmatrix}
1 & 2 & 2 & 2 \\
0 & 0 & 2 & 4 \\
0 & 0 & 0 & 0
\end{bmatrix}$

row-3 contains only zeros because original it was a linear combination
of some eariler rows (row-1 and row-2)

**reduced row echelon form has 0 above and below pivots**

$R = \begin{bmatrix}
1 & 2 & 0 & -2 \\
0 & 0 & 1 & 2 \\
0 & 0 & 0 & 0
\end{bmatrix}$



In [10]:
import pylinearalgebra.mattools as mt

# source:
# https://stackoverflow.com/questions/7664246/python-built-in-function-to-do-matrix-reduction

R, i_pivots = mt.rref([
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
])

# Matlab also offers rref() function
mt.pprint(R)

⎡  1   2   0  -2 ⎤
⎢  0   0   1   2 ⎥
⎣  0   0   0   0 ⎦


#### information encoded in the RREF

1) pivot rows, pivot columns, which form an $I$ (which left a "free"
part of the matrix, $F$)

2) cleaned-up free columns

3) $Rx = 0$

(recap: from $Ax = 0$ to $Ux = 0$, to $Rx = 0$)

#### how to interpret RREF

a typical RREF 

$R = \begin{bmatrix}
I & F \\
0* & 0*
\end{bmatrix}$

(the zero-rows are optional)

that contains $r$ pivot rows, and $r$ pivot columns, and
$n - r$ free columns

construct a nullspace solution out of RREF

**using purely the matrix multiplication rules 
(see the block-multiplication
breakdown example in the previous chapter:
18.06_03_multiplication_and_inverse)**

I can derive a formula


$\begin{align}
RN = 0
\\
N = \begin{bmatrix}
-F \\ I
\end{bmatrix}
\end{align}$

NOTE: this might not a generic enough formula (as I found out from implementing this in Python using
sympy and numpy), later on there is a `SVD` decomposition which could help

at the time of writing, the `solve_rn()` function in `mattools` produce the right answer but also some
noises (like one extra column)

In [1]:
# implement this formula,
# using sympy's Matrix class
# Matlab appears to have a command for computing N

import pylinearalgebra.mattools as mt
A = [
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
]
solutions = mt.solve_rn(A)
for v in solutions:
    mt.pprint(v)

⎡   -2⎤
⎢    0⎥
⎢ 1.00⎥
⎣ 0.00⎦
⎡    2⎤
⎢   -2⎥
⎢ 0.00⎥
⎣ 1.00⎦


#### back substitution on the reduced form

$\begin{align}
Rx = 0
\\
\begin{bmatrix}
I & F
\end{bmatrix} \ 
\begin{bmatrix}
x_{pivot} \\ x_{free}
\end{bmatrix}
= 0 \\
Rx = 0 = I x_{pivot} + F x_{free}
\\
x_{pivot} + Fx_{free} = 0
\end{align}$


if I choose to assign $I$ to $x_{free}$, then the whole equation
can be simplified to:

$x_{pivot} + F = 0\\
x_{pivot} = -F$

In [5]:
# transpose A and get a 3 x 4 matrix
# the algorithm still applies

import pylinearalgebra.mattools as mt
A = [
    [1, 2, 2, 2],
    [2, 4, 6, 8],
    [3, 6, 8, 10]
]
AT = mt.T(A)
solutions = mt.solve_rn(AT)
for v in solutions:
    mt.pprint(v)


⎡   -1⎤
⎢   -1⎥
⎢ 1.00⎥
⎣ 0.00⎦


# Recitation and Exercises

In [3]:
# if I have a R3 (3-d space) and a single constraint: 
# x - 5y + 2z = 9
# it forms a subspace with one less degree of freedom
# it is a plane

# given two such planes:
# x - 5y + 2z = 9
# x - 5y + 2z = 0
# there is not a line that exists in both planes
# i.e. there is no such as `{x, y, z}` that specifies
# both equations
# therefore these two planes are parallel to each other
# there is no intersection.

import sympy

rows = [
    [1, -5, 2, 9],
    [1, -5, 2, 0]
]

# observe the RREF of the augmented matrix
# with RHS plugged in, there is no solution
A = sympy.Matrix(rows)
R, _ = A.rref()
R

Matrix([
[1, -5, 2, 0],
[0,  0, 0, 1]])

In [8]:
# problem 7.1
# find the RREF
# what is the rank
# find any special solutions to Ax = 0

# recall:
# https://en.wikipedia.org/wiki/Rank_(linear_algebra)

# the rank of a matrix A is the dimension of the vector space 
# generated (or spanned) by its columns.

# The column rank of A is the dimension of the column space of A, 
# while the row rank of A is the dimension of the row space of A. 

import sympy 

rows = [
    [1,  5,  7,  9],
    [0,  4,  1,  7],
    [2, -2, 11, -3]
]
A= sympy.Matrix(rows)
R, i_pivots = A.rref()
R


Matrix([
[1, 0, 23/4, 1/4],
[0, 1,  1/4, 7/4],
[0, 0,    0,   0]])

In [9]:
i_pivots

(0, 1)

In [14]:
F_neg = R[[0, 1], [2, 3]] * -1
F_neg

Matrix([
[-23/4, -1/4],
[ -1/4, -7/4]])

In [15]:
I = R[[0, 1], [0, 1]]
I

Matrix([
[1, 0],
[0, 1]])

In [16]:
# the special solutions for the nullspace

[
    [-23/4.0, -1/4.0, 1, 0],
    [-1/4.0, -7/4.0, 0, 1]
]

[[-5.75, -0.25, 1, 0], [-0.25, -1.75, 0, 1]]

In [27]:
# problem 7.2

# for A1 and A2 so that rank(A1B) = 1 and rank(A2B) = 0 for B

import sympy

B = sympy.Matrix([
    [1, 1],
    [1, 1]
])
A1 = sympy.Matrix([
    [2, 2],
    [2, 2]
])
A1B = A1 * B
print(A1B)
print(A1B.rank())

A2 = sympy.Matrix([
    [-1, 1],
    [1, -1]
])
A2B = A2 * B
print(A2B)
print(A2B.rank())

Matrix([[4, 4], [4, 4]])
1
Matrix([[0, 0], [0, 0]])
0
